In [2]:
# Import required libraries
import numpy as np
import math
import datetime as dt
import pandas as pd
import pathlib

In [3]:
# Load data
# get relative data folder
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
PIDAR = pd.read_csv(DATA_PATH.joinpath("pidar.csv"))
AT = pd.read_csv(DATA_PATH.joinpath("DS_Asistencia_Tecnica.csv"))
EVA = pd.read_csv(DATA_PATH.joinpath('EVA_cultivos_modificado.csv'))
url = 'https://raw.githubusercontent.com/ds4a-team-envigado/project/master/Dashboard/VERSION%202/data/AP_SABANAGENERAL.csv'
SG = pd.read_csv(url, index_col=0,sep=";",encoding='latin-1')
dfs = [PIDAR,AT,EVA,SG]

In [4]:
# #Agrupar por departamtento cada una de las columnas
dff = EVA.groupby(['DEPARTAMENTO','YEAR','CADENA_PRODUCTIVA_ADR']).sum().reset_index()
dff.head()

,DEPARTAMENTO,YEAR,CADENA_PRODUCTIVA_ADR,COD_DEP,COD_MUN,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,NIVEL,ID_ADR
0,AMAZONAS,2006,ARROZ,273,274339,11,11,16,4.35,9,37527
1,AMAZONAS,2006,MAIZ,455,457151,204,203,468,7.86,15,62520
2,AMAZONAS,2007,ACHICORIA,182,182541,6,6,6,2.00,6,21610
3,AMAZONAS,2007,AHUYAMA,91,91540,10,10,38,3.80,3,11213
4,AMAZONAS,2007,AJI,182,182541,28,28,35,2.50,6,22418


In [6]:
AT.head()

,Unnamed: 0,Id_Usuario,Sexo,Tipo_Poblacional,Discapacidad,Departamento,Municipio,Vereda,Predio,Cadena_Productiva,...,NA4,C1A5,C2A5,C3A5,C4A5,NA5,CLAS_GRAL,CADENA_PRODUCTIVA_ADR,ID_ADR,Tipo de proyecto
0,0,1,M,NINGUNA,NO,BOLIVAR,13838,CARABALI,PIE DE MONTE,MAIZ,...,3,3.0,3.0,3.0,3.0,3,2,MAIZ,12504.0,Asistencia Tecnica
1,3,4,M,NINGUNA,NO,BOLIVAR,13647,PATON,PATON,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
2,8,9,M,NINGUNA,NO,BOLIVAR,13248,La Enea,Sin nombre,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
3,10,11,M,NINGUNA,NO,BOLIVAR,13248,La Enea,Sin nombre,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
4,13,14,M,NINGUNA,NO,BOLIVAR,13780,PATICO,LAS PALMAS,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica


In [8]:
AT.groupby(['Departamento','CADENA_PRODUCTIVA_ADR'])[['Id_Usuario']].sum().reset_index()

,Departamento,CADENA_PRODUCTIVA_ADR,Id_Usuario
0,BOLIVAR,ACUICULTURA,258307
1,BOLIVAR,AGRICOLA,187448
2,BOLIVAR,AGROPECUARIO,74535
3,BOLIVAR,APICULTURA,38829
4,BOLIVAR,ARROZ,261676
5,BOLIVAR,AVICULTURA,2314
6,BOLIVAR,CACAO,150858
7,BOLIVAR,CITRICOS,996
8,BOLIVAR,COMERCIALIZACION,613
9,BOLIVAR,GANADERIA,1977993


In [9]:
SG.head()

,AP_Porcentaje_Avance,AP_Beneficiarios_Definitivos,AP_Cant_Afrocolombiano,AP_Cant_Campesinos,AP_Cant_Mujeres,AP_Cant_Indigenas,AP_Cant_Hombre,AP_Cant_MujerRural,AP_Raizal,AP_Cant_Victimas,AP_Sector_Productivo,AP_ID_ADR,AP_CADENA_PRODUCTIVA_ADR,AP_Nombre_Proyecto,AP_Fecha_Resolución,AP_Departamento
AP_Numero_BP,,,,,,,,,,,,,,,,
33,100,12,0,12,1,0,11,0,0,0,Agricola,1,AGRICOLA,Fortalecer la cadena productiva de la caña pan...,12/07/2017,ANTIOQUIA
23,50,25,0,25,5,0,20,0,0,23,Agricola,1,AGRICOLA,Fortalecer la cadena productiva de café y plát...,28/08/2017,ANTIOQUIA
89,100,100,0,100,22,0,78,0,0,13,Agricola,1,AGRICOLA,"Una hectárea para la vida, una hectárea para l...",23/03/2018,ANTIOQUIA
183,"16,04",20,0,20,6,0,14,0,0,4,Agricola,1,AGRICOLA,Fortalecimiento de la actividad agrícola en el...,17/07/2018,ANTIOQUIA
348,"78,96",30,0,30,30,0,0,0,0,16,Agricola,1,AGRICOLA,Fortalecimiento de Mujeres cabeza de familia a...,17/07/2018,ANTIOQUIA
